<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Streaming-Visualization-with-PyQtGraph" data-toc-modified-id="Streaming-Visualization-with-PyQtGraph-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Streaming Visualization with PyQtGraph</a></span><ul class="toc-item"><li><span><a href="#Basics" data-toc-modified-id="Basics-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Basics</a></span><ul class="toc-item"><li><span><a href="#QApplication.processEvents()" data-toc-modified-id="QApplication.processEvents()-1.1.1"><span class="toc-item-num">1.1.1&nbsp;&nbsp;</span><code>QApplication.processEvents()</code></a></span></li><li><span><a href="#QTimer" data-toc-modified-id="QTimer-1.1.2"><span class="toc-item-num">1.1.2&nbsp;&nbsp;</span><code>QTimer</code></a></span></li><li><span><a href="#OO-Approach" data-toc-modified-id="OO-Approach-1.1.3"><span class="toc-item-num">1.1.3&nbsp;&nbsp;</span>OO Approach</a></span></li></ul></li><li><span><a href="#Essential-Widgets-&amp;-Features" data-toc-modified-id="Essential-Widgets-&amp;-Features-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Essential Widgets &amp; Features</a></span><ul class="toc-item"><li><span><a href="#Flowchart" data-toc-modified-id="Flowchart-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>Flowchart</a></span></li><li><span><a href="#Gradient-Editors" data-toc-modified-id="Gradient-Editors-1.2.2"><span class="toc-item-num">1.2.2&nbsp;&nbsp;</span>Gradient Editors</a></span></li><li><span><a href="#ImageItem" data-toc-modified-id="ImageItem-1.2.3"><span class="toc-item-num">1.2.3&nbsp;&nbsp;</span>ImageItem</a></span></li><li><span><a href="#ImageView" data-toc-modified-id="ImageView-1.2.4"><span class="toc-item-num">1.2.4&nbsp;&nbsp;</span>ImageView</a></span></li></ul></li><li><span><a href="#Application-to-heatflow-visualization" data-toc-modified-id="Application-to-heatflow-visualization-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Application to heatflow visualization</a></span></li><li><span><a href="#UI-embedded-Widget" data-toc-modified-id="UI-embedded-Widget-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>UI-embedded Widget</a></span></li></ul></li></ul></div>

# Streaming Visualization with PyQtGraph

## Basics

Pyqtgraph is highly performant when it comes to drawing / updating plots quickly. How to achieve realtime plotting is highly dependent on the details and control flow in a given application.

The most common ways are:

### `QApplication.processEvents()`

1. Plot data within a loop that makes calls to `QApplication.processEvents()`.

In [ ]:
%gui qt5
import numpy as np
import pyqtgraph as pg
from time import sleep
from math import sin

x = [i/10 for i in range(200)]
y = [sin(i)+np.random.rand() for i in x]

# method 1
pw = pg.plot()
while True:
    x.append((x[-1]*10+1)/10)
    y.append(sin(x[-1])+np.random.rand())
    pw.plot(x[-200:], y[-200:], clear=True)
    pg.QtGui.QApplication.processEvents()
    sleep(0.05)

KeyboardInterrupt: 

### `QTimer`

2. Use a `QTimer` to make repeated calls to a function that updates the plot.

In [1]:
%gui qt5
import numpy as np
import pyqtgraph as pg
from time import sleep
from math import sin

x = [i/10 for i in range(200)]
y = [sin(i)+np.random.rand() for i in x]

# method 2
pw = pg.plot()
timer = pg.QtCore.QTimer()

def update():
    x.append((x[-1]*10+1)/10)
    y.append(sin(x[-1])+np.random.rand())
    pw.plot(x[-200:], y[-200:], clear=True)
    
timer.timeout.connect(update)
timer.start(50) # <----- 50 milliseconds between loops

### OO Approach

In [1]:
from pyqtgraph.Qt import QtGui, QtCore
import pyqtgraph as pg

import collections
import random
import time
import math
import numpy as np

class DynamicPlotter():
    def __init__(self, sampleinterval=0.1, timewindow=10., size=(600,350)):
        # Data stuff
        self._interval = int(sampleinterval*1000)
        self._bufsize = int(timewindow/sampleinterval)
        self.databuffer = collections.deque([0.0]*self._bufsize, self._bufsize)
        self.x = np.linspace(0., timewindow, self._bufsize)
        self.y = np.zeros(self._bufsize, dtype=np.float)
        # PyQtGraph stuff
        self.app = QtGui.QApplication([])
        self.plt = pg.plot(title='Dynamic Plotting with PyQtGraph')
        self.plt.resize(*size)
        self.plt.showGrid(x=True, y=True)
        self.plt.setLabel('left', 'amplitude', 'V')
        self.plt.setLabel('bottom', 'time', 's')
        self.curve = self.plt.plot(self.x, self.y, pen=(255,0,0))
        # QTimer
        self.timer = QtCore.QTimer()
        self.timer.timeout.connect(self.updateplot)
        self.timer.start(self._interval)

    def getdata(self):
        frequency = 0.5
        noise = random.normalvariate(0., 1.)
        new = 10.*math.sin(time.time()*frequency*2*math.pi) + noise
        return new

    def updateplot(self):
        self.databuffer.append( self.getdata() )
        self.y[:] = self.databuffer
        self.curve.setData(self.x, self.y)
        self.app.processEvents()

    def run(self):
        self.app.exec_()

if __name__ == '__main__':
    
    try:
        m.app = 0
        m = 0
    except:
        pass
    m = DynamicPlotter(sampleinterval=0.05, timewindow=10.)
    m.run()

## Essential Widgets & Features

Below are examples copied directly from documentation.

### Flowchart

In [1]:
from pyqtgraph.flowchart import Flowchart
from pyqtgraph.Qt import QtGui, QtCore
import pyqtgraph as pg
import numpy as np
import pyqtgraph.metaarray as metaarray

app = QtGui.QApplication([])

## Create main window with grid layout
win = QtGui.QMainWindow()
win.setWindowTitle('pyqtgraph example: Flowchart')
cw = QtGui.QWidget()
win.setCentralWidget(cw)
layout = QtGui.QGridLayout()
cw.setLayout(layout)

## Create flowchart, define input/output terminals
fc = Flowchart(terminals={
    'dataIn': {'io': 'in'},
    'dataOut': {'io': 'out'}    
})
w = fc.widget()

## Add flowchart control panel to the main window
layout.addWidget(fc.widget(), 0, 0, 2, 1)

## Add two plot widgets
pw1 = pg.PlotWidget()
pw2 = pg.PlotWidget()
layout.addWidget(pw1, 0, 1)
layout.addWidget(pw2, 1, 1)

win.show()

## generate signal data to pass through the flowchart
data = np.random.normal(size=1000)
data[200:300] += 1
data += np.sin(np.linspace(0, 100, 1000))
data = metaarray.MetaArray(data, info=[{'name': 'Time', 'values': np.linspace(0, 1.0, len(data))}, {}])

## Feed data into the input terminal of the flowchart
fc.setInput(dataIn=data)

## populate the flowchart with a basic set of processing nodes. 
## (usually we let the user do this)
plotList = {'Top Plot': pw1, 'Bottom Plot': pw2}

pw1Node = fc.createNode('PlotWidget', pos=(0, -150))
pw1Node.setPlotList(plotList)
pw1Node.setPlot(pw1)

pw2Node = fc.createNode('PlotWidget', pos=(150, -150))
pw2Node.setPlot(pw2)
pw2Node.setPlotList(plotList)

fNode = fc.createNode('GaussianFilter', pos=(0, 0))
fNode.ctrls['sigma'].setValue(5)
fc.connectTerminals(fc['dataIn'], fNode['In'])
fc.connectTerminals(fc['dataIn'], pw1Node['In'])
fc.connectTerminals(fNode['Out'], pw2Node['In'])
fc.connectTerminals(fNode['Out'], fc['dataOut'])



## Start Qt event loop unless running in interactive mode or using pyside.
if __name__ == '__main__':
    import sys
    if (sys.flags.interactive != 1) or not hasattr(QtCore, 'PYQT_VERSION'):
        QtGui.QApplication.instance().exec_()

c:\program files\python37\lib\site-packages\pyqtgraph\functions.py:1276: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  filtered = filtered[sl]


### Gradient Editors

In [1]:
import numpy as np
from pyqtgraph.Qt import QtGui, QtCore
import pyqtgraph as pg


app = QtGui.QApplication([])
mw = pg.GraphicsView()
mw.resize(800,800)
mw.show()

#ts = pg.TickSliderItem()
#mw.setCentralItem(ts)
#ts.addTick(0.5, 'r')
#ts.addTick(0.9, 'b')

ge = pg.GradientEditorItem()
mw.setCentralItem(ge)


## Start Qt event loop unless running in interactive mode.
if __name__ == '__main__':
    import sys
    if (sys.flags.interactive != 1) or not hasattr(QtCore, 'PYQT_VERSION'):
        QtGui.QApplication.instance().exec_()

In [1]:
import pyqtgraph as pg
from pyqtgraph.Qt import QtCore, QtGui
import numpy as np



app = QtGui.QApplication([])
w = QtGui.QMainWindow()
w.show()
w.setWindowTitle('pyqtgraph example: GradientWidget')
w.setGeometry(10, 50, 400, 400)
cw = QtGui.QWidget()
w.setCentralWidget(cw)

l = QtGui.QGridLayout()
l.setSpacing(0)
cw.setLayout(l)

w1 = pg.GradientWidget(orientation='top')
w2 = pg.GradientWidget(orientation='right', allowAdd=False)
#w2.setTickColor(1, QtGui.QColor(255,255,255))
w3 = pg.GradientWidget(orientation='bottom')
w4 = pg.GradientWidget(orientation='left')
w4.loadPreset('spectrum')
label = QtGui.QLabel("""
- Click a triangle to change its color
- Drag triangles to move
- Click in an empty area to add a new color
    (adding is disabled for the right-side widget)
- Right click a triangle to remove
""")

l.addWidget(w1, 0, 1)
l.addWidget(w2, 1, 2)
l.addWidget(w3, 2, 1)
l.addWidget(w4, 1, 0)
l.addWidget(label, 1, 1)


## Start Qt event loop unless running in interactive mode or using pyside.
if __name__ == '__main__':
    import sys
    if (sys.flags.interactive != 1) or not hasattr(QtCore, 'PYQT_VERSION'):
        QtGui.QApplication.instance().exec_()

### ImageItem

For plotting images and 2D arrays

In [1]:
from pyqtgraph.Qt import QtCore, QtGui
import numpy as np
import pyqtgraph as pg
import pyqtgraph.ptime as ptime

app = QtGui.QApplication([])

## Create window with GraphicsView widget
win = pg.GraphicsLayoutWidget()
win.show()  ## show widget alone in its own window
win.setWindowTitle('pyqtgraph example: ImageItem')
view = win.addViewBox()

## lock the aspect ratio so pixels are always square
view.setAspectLocked(True)

## Create image item
img = pg.ImageItem(border='w')
view.addItem(img)

## Set initial view bounds
view.setRange(QtCore.QRectF(0, 0, 600, 600))

## Create random image
data = np.random.normal(size=(15, 600, 600), loc=1024, scale=64).astype(np.uint16)
i = 0

updateTime = ptime.time()
fps = 0

def updateData():
    global img, data, i, updateTime, fps

    ## Display the data
    img.setImage(data[i])
    i = (i+1) % data.shape[0]

    QtCore.QTimer.singleShot(1, updateData)
    now = ptime.time()
    fps2 = 1.0 / (now-updateTime)
    updateTime = now
    fps = fps * 0.9 + fps2 * 0.1
    
    #print "%0.1f fps" % fps
    

updateData()

## Start Qt event loop unless running in interactive mode.
if __name__ == '__main__':
    import sys
    if (sys.flags.interactive != 1) or not hasattr(QtCore, 'PYQT_VERSION'):
        QtGui.QApplication.instance().exec_()


### ImageView

For Scientific Image Analysis

In [1]:
import numpy as np
from pyqtgraph.Qt import QtCore, QtGui
import pyqtgraph as pg

# Interpret image data as row-major instead of col-major
pg.setConfigOptions(imageAxisOrder='row-major')

app = QtGui.QApplication([])

## Create window with ImageView widget
win = QtGui.QMainWindow()
win.resize(800,800)
imv = pg.ImageView()
win.setCentralWidget(imv)
win.show()
win.setWindowTitle('pyqtgraph example: ImageView')

## Create random 3D data set with noisy signals
img = pg.gaussianFilter(np.random.normal(size=(200, 200)), (5, 5)) * 20 + 100
img = img[np.newaxis,:,:]
decay = np.exp(-np.linspace(0,0.3,100))[:,np.newaxis,np.newaxis]
data = np.random.normal(size=(100, 200, 200))
data += img * decay
data += 2

## Add time-varying signal
sig = np.zeros(data.shape[0])
sig[30:] += np.exp(-np.linspace(1,10, 70))
sig[40:] += np.exp(-np.linspace(1,10, 60))
sig[70:] += np.exp(-np.linspace(1,10, 30))

sig = sig[:,np.newaxis,np.newaxis] * 3
data[:,50:60,30:40] += sig


## Display the data and assign each frame a time value from 1.0 to 3.0
imv.setImage(data, xvals=np.linspace(1., 3., data.shape[0]))

## Set a custom color map
colors = [
    (0, 0, 0),
    (45, 5, 61),
    (84, 42, 55),
    (150, 87, 60),
    (208, 171, 141),
    (255, 255, 255)
]
cmap = pg.ColorMap(pos=np.linspace(0.0, 1.0, 6), color=colors)
imv.setColorMap(cmap)

## Start Qt event loop unless running in interactive mode.
if __name__ == '__main__':
    import sys
    if (sys.flags.interactive != 1) or not hasattr(QtCore, 'PYQT_VERSION'):
        QtGui.QApplication.instance().exec_()


c:\program files\python37\lib\site-packages\pyqtgraph\functions.py:1276: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  filtered = filtered[sl]
c:\program files\python37\lib\site-packages\pyqtgraph\imageview\ImageView.py:588: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  data = data[sl]
c:\program files\python37\lib\site-packages\pyqtgraph\imageview\ImageView.py:588: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as a

## Application to heatflow visualization

In [1]:
import numpy as np
from numba import jit, jitclass, int64, float64

spec = [('w', int64),
        ('h', int64),
        ('D', int64)]

@jitclass(spec)
class hotplate:
    def __init__(self, w=10, h=20, D=4):
        self.w = w
        self.h = h
        self.D = D

spec = [('boundary_conditions', float64[:]),
        ('relaxation_guess',float64),
        ('ds',float64),
        ('maxiter',int64),
        ('dx', float64),
        ('dy', float64),
        ('dx2', float64),
        ('dy2', float64),
        ('nx', int64),
        ('ny', int64),
        ('dt', float64),
        ('u0', float64[:,:]),
        ('u', float64[:,:]),
        ('D', float64)]
@jitclass(spec)
class steadystate:
    def __init__(self,
                 hotplate,
                 boundary_conditions=[100.,0.,0.,.0],
                 relaxation_guess=30.,
                 ds = 0.1,
                 maxiter=500):
        self.boundary_conditions = boundary_conditions
        self.relaxation_guess = relaxation_guess
        self.ds = 0.1
        self.maxiter = maxiter
        self.D = hotplate.D # not perfect, but okay


        # SEE: second derivatives of spatially-dependent terms
        self.dx, self.dy = ds, ds
        self.dx2, self.dy2 = (self.dx*self.dx), (self.dy*self.dy)
        self.dt = self.dx2*self.dy2 / (2 * hotplate.D * (self.dx2 + self.dy2))

        # number of discrete points in each cardinal direction
        self.nx, self.ny = int(hotplate.w/self.dx), int(hotplate.h/self.dy)

        # Initialize the array we'll do calculations on
        # and set the interior value to Tguess
        self.u0 = np.empty((self.nx, self.ny), dtype=np.float64)

        for i in range(self.nx):
            for j in range(self.ny):
                self.u0[i, j] = relaxation_guess

        # Set boundary conditions for the steady state before t=0
        self.u0[:, (self.ny-1):] = boundary_conditions[0]
        self.u0[:, :1] = boundary_conditions[1]
        self.u0[:1, :] = boundary_conditions[-2]
        self.u0[(self.nx-1):, :] = boundary_conditions[-1]
        
    def calculate(self):
        # Compute the steady state distribution, where u0 = Steady-State Temperature Matrix
        for iteration in range(0, self.maxiter):
            for i in range(1, self.nx-1):
                for j in range(1, self.ny-1):
                    self.u0[i, j] = (0.25) * (self.u0[i+1][j] + self.u0[i-1][j] +
                                        self.u0[i][j+1] + self.u0[i][j-1])

spec = []
@jitclass(spec)
class hotboundaries:
    def __init__(self,
                 steadyplate, # pass in a hotplate with an initialized and/or state
                 boundary_conds=[0.0,0.0,0.0,0.0]):
        
        # New initial conditions
        Ttop_new = boundary_conds[0]
        Tbottom_new = boundary_conds[1]
        Tleft_new = boundary_conds[-2]
        Tright_new = boundary_conds[-1]

        # Set new boundary conditions in the steady-state matrix
        steadyplate.u0[:, :5] = Tbottom_new
        steadyplate.u0[(steadyplate.nx-5):, :] = Tright_new
        steadyplate.u0[:5, :] = Tleft_new
        steadyplate.u0[:, (steadyplate.ny-5):] = Ttop_new

    
# produce a hot spot on the plate and observe thermal diffusion
# circle of radius r, centred at c=(cx,cy)
spec = []
@jitclass(spec)
class hotspot:
    def __init__(self,
                 steadyplate, # pass in a hotplate with an initialized and/or steady state
                 temperature = 200, # temperature of spot
                 r = 1, # radius of circle
                 c = (0, 0) # position of center of circle on plate
                 ): 
        r2 = r*r # radius-squared
        for i in range(steadyplate.nx):
            for j in range(steadyplate.ny):
                p2 = (i * steadyplate.dx - c[0])**2 + (j * steadyplate.dy - c[1])**2
                if p2 < r2:
                    steadyplate.u0[i, j] = temperature
                    
spec = [('tsteps', int64),
        ('D', float64),
        ('dx2', float64),
        ('dy2', float64),
        ('nx', float64),
        ('ny', float64),
        ('dt', float64),
        ('u0', float64[:,:]),
        ('u', float64[:,:]),
        ('frames', float64[:,:,:])]
@jitclass(spec)
class heatflow:    
    def do_timestep(self):
        # Propagate with forward-difference in time, central-difference in space
        self.u[1:-1, 1:-1] = self.u0[1:-1, 1:-1] + self.D * self.dt * (
            (self.u0[2:, 1:-1] - 2*self.u0[1:-1, 1:-1] + self.u0[:-2, 1:-1])/self.dx2
            + (self.u0[1:-1, 2:] - 2*self.u0[1:-1, 1:-1] + self.u0[1:-1, :-2])/self.dy2)

        self.u0 = self.u
        
    def __init__(self,
                 steadyplate # pass in a hotplate entity with an initialized and/or steady state
                ):
        self.u = steadyplate.u0.copy()
        self.u0 = steadyplate.u0
        self.dt = steadyplate.dt
        self.dx2 = steadyplate.dx2
        self.dy2 = steadyplate.dy2
        self.nx = steadyplate.nx
        self.ny = steadyplate.ny
        self.D = steadyplate.D
    
    def calculate(self,
                  tsteps, # number of time steps
                  snapshots # on which timesteps should snapshots (frames) be save for visualizing?
                  ):
        # create an array of frames for visualizing the heatflow
        self.frames = np.empty((int(self.nx), int(self.ny), len(snapshots)), dtype=np.float64)
        
        # index for saving frames on the z-axis if the self.frames
        i = 0
        
        # perform timestep calculations and save the frames
        for n in range(tsteps): 
            self.do_timestep()
            
            # save the specified frames
            if n in snapshots:
                self.frames[:,:,i] = self.u0.copy()
                i+=1

In [2]:
import sys
from pyqtgraph.Qt import QtCore, QtGui
import numpy as np
import pyqtgraph as pg
import pyqtgraph.ptime as ptime
from matplotlib import cm

app = QtGui.QApplication([])
pg.setConfigOption('background', 'k')
pg.setConfigOption('foreground', 'w')

## Create window with Graphics widget
win = pg.GraphicsWindow()
win.show() 
win.setWindowTitle('Heatflow Animation')
view = win.addViewBox()
view.setAspectLocked(True) # so pixels stay square

img = pg.ImageItem(border='w') # this displays the heatflow
view.addItem(img)
view.setRange(QtCore.QRectF(0, 0, 600, 600)) # initial view bounds

## Display FPS and iteration count
fps_txt = pg.TextItem(text='FPS: 0.0', color=(200, 200, 200), 
                      anchor=(0, 0), border=None, fill=None)
iter_txt = pg.TextItem(text='Timestep: 0.0', color=(200, 200, 200), 
                       anchor=(0, 0), border=None, fill=None)

fps_txt.setFont(pg.Qt.QtGui.QFont("times", 10))
iter_txt.setFont(pg.Qt.QtGui.QFont("arial", 10))

fps_txt.setPos(0, 675)
iter_txt.setPos(0, 650)

view.addItem(fps_txt)
view.addItem(iter_txt)

## Generate steady state object and initialize heatflow
b = steadystate(hotplate(60, 60, 4),
                boundary_conditions=np.array([100.,0.,0.,.0]),
                relaxation_guess=30.,
                ds = 0.1,
                maxiter=500)
b.calculate()

    # Set IC's after steady state is reached but before heatflow occurs
hotboundaries(b, np.array([100.,60.,60.,60.]))
hotspot(b, 80., 25, (30, 30))
hotspot(b, 50., 15, (30, 30))
hotspot(b, 99., 5, (30, 30))
b = heatflow(b)

## Colormap the values of the heatflow matrix
colormap = cm.get_cmap("inferno")
colormap._init()
   # Lookup Table (lut); converts mpl.cm from 0-1 to 0-255 for Qt
lut = (colormap._lut * 255).view(np.ndarray)  
img.setLookupTable(lut) # apply the colormap

## Track number of iterations and fps
updateTime = ptime.time()
fps = 0
i = 0

def updateHeatflow():
    global img, data, i, updateTime, fps

    b.calculate(1, [0])
    
    ## Display the data
    img.setImage(b.u0)
    #set the levels to the min and max we desire on our scale
    img.setLevels([0,100]) 

    #QtCore.QTimer.singleShot(1, updateHeatflow)
    now = ptime.time()
    fps2 = 1.0 / (now-updateTime)
    updateTime = now
    fps = fps * 0.9 + fps2 * 0.1
    
    fps_txt.setText('FPS: {}'.format(round(fps, 2)), (255,0,0))
    iter_txt.setText('Timestep: {}'.format(i), (255,0,0))
    
    i+=1


#updateHeatflow()
timer1 = pg.QtCore.QTimer()
timer1.timeout.connect(updateHeatflow)
timer1.start(0)

''' '''   
p1 = win.addPlot(labels =  {'left':'Also Random', 'bottom':'Random'})
data1 = [np.random.normal() for i in range(10)]
curve1 = p1.plot(data1, pen=(7,7))
i1 = 0

def updateGraph():
    global data1, curve1, i1

    data1[:-1] = data1[1:]
    data1[-1] = np.random.normal()
    i1 += 1

    curve1.setData(data1)
    curve1.setPos(i1,0)

def update():
    updateHeatflow()
    updateGraph()

timer0 = pg.QtCore.QTimer()
timer0.timeout.connect(updateGraph)
timer0.start(100)
'''  '''  


## Start Qt event loop unless running in interactive mode.
if __name__ == '__main__':
    QtGui.QApplication.instance().exec_()

## UI-embedded Widget

By creating an object (a class) for our widget, we can embed it in a GUI via QT's GUI Creator.


See this stackexchange thread for a direct tutorial: https://stackoverflow.com/questions/45872255/embed-a-pyqtgraph-plot-into-a-qt-ui

In [1]:
import sys
import pyqtgraph as pg
from pyqtgraph.Qt import QtCore, QtGui
import numpy as np
from heatflow import *
from matplotlib import cm

class  CustomWidget(pg.GraphicsWindow):
    pg.setConfigOption('background', 'k')
    pg.setConfigOption('foreground', 'w')
    ptr1 = 0
    def __init__(self, parent=None, **kargs):
        pg.GraphicsWindow.__init__(self, **kargs)
        self.setParent(parent)
        self.setWindowTitle('Heatflow Animation')

        view = self.addViewBox()
        view.setAspectLocked(True) # so pixels stay square

        self.img = pg.ImageItem(border='w') # this displays the heatflow
        view.addItem(self.img)
        view.setRange(QtCore.QRectF(0, 0, 600, 600)) # initial view bounds
        
        ## Display FPS and iteration count
        self.fps_txt = pg.TextItem(text='FPS: 0.0', color=(200, 200, 200), 
                              anchor=(0, 0), border=None, fill=None)
        self.iter_txt = pg.TextItem(text='Timestep: 0.0', color=(200, 200, 200), 
                               anchor=(0, 0), border=None, fill=None)

        self.fps_txt.setFont(pg.Qt.QtGui.QFont("times", 10))
        self.iter_txt.setFont(pg.Qt.QtGui.QFont("arial", 10))

        self.fps_txt.setPos(0, 675)
        self.iter_txt.setPos(0, 650)

        view.addItem(self.fps_txt)
        view.addItem(self.iter_txt)

        ## Generate steady state object and initialize heatflow
        b = steadystate(hotplate(60, 60, 4),
                        boundary_conditions=np.array([100.,0.,0.,.0]),
                        relaxation_guess=30.,
                        ds = 0.1,
                        maxiter=500)
        b.calculate()

            # Set IC's after steady state is reached but before heatflow occurs
        hotboundaries(b, np.array([100.,60.,60.,60.]))
        hotspot(b, 80., 25, (30, 30))
        hotspot(b, 50., 15, (30, 30))
        hotspot(b, 99., 5, (30, 30))
        self.b = heatflow(b)

        ## Colormap the values of the heatflow matrix
        colormap = cm.get_cmap("inferno")
        colormap._init()
           # Lookup Table (lut); converts mpl.cm from 0-1 to 0-255 for Qt
        lut = (colormap._lut * 255).view(np.ndarray)  
        self.img.setLookupTable(lut) # apply the colormap

        ## Track number of iterations and fps
        self.updateTime = pg.ptime.time()
        self.fps = 0
        self.i = 0

        timer = pg.QtCore.QTimer(self)
        timer.timeout.connect(self.update)
        timer.start(0)
        
    def update(self):
        self.b.calculate(1, [0])

        ## Display the data
        self.img.setImage(self.b.u0)
        # set the max an min levels to interpolate
        self.img.setLevels([0,100]) 

        now = pg.ptime.time()
        fps2 = 1.0 / (now - self.updateTime)
        self.updateTime = now
        self.fps = self.fps * 0.9 + fps2 * 0.1

        self.fps_txt.setText('FPS: {}'.format(round(self.fps, 2)), (255,0,0))
        self.iter_txt.setText('Timestep: {}'.format(self.i), (255,0,0))

        self.i+=1

if __name__ == '__main__':
    w = CustomWidget()
    w.show()
    QtGui.QApplication.instance().exec_()